## Notebook 2: Apprentissage automatique, Regression: Polution CO2

#### Importation des librairies

In [149]:
import pandas as pd
import numpy as np
#-------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer
from sklearn.metrics import *
#-------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer

#--------------------------------------------------------
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

#--------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
#------------------------------------------------
from xgboost import XGBRegressor

#-----------------------------------------------------------
from sklearn.model_selection import GridSearchCV
#-----------------------------------------------------------
from joblib import dump,load

## Checklist

Depuis 2001, **l’ADEME** acquiert tous les ans ces données auprès de **l’Union Technique de l’Automobile du motocycle et du Cycle UTAC** (en charge de l’homologation des véhicules avant leur mise en vente) en accord avec le ministère du développement durable.
Pour chaque véhicule les données d’origine (transmises par l’Utac) sont les suivantes :

* **Les consommations de carburant**

* **Les émissions de dioxyde de carbone (CO2)**

* **Les émissions des polluants de l’air** (réglementés dans le cadre de la norme Euro)

* **L’ensemble des caractéristiques techniques des véhicules** (gammes, marques, modèles, n° de CNIT, type d’énergie ...)



# L'inventaire des variables pertinentes:

Les données comprenent des variables pertinentes suivantes:

* **lib_mrq_utac**: La marque, il y'a 12.

* **lib_mod**: Le modèle commerciale, il y'a 20.

* **cod_cbr**: Le type de carburant, il y a 5.

* **hybride**: Information permettant d’identifier les véhicules hybrides (O/N)

* **puiss_max** : Puissance maximale

* **typ_boite_nb_rapp**: Type boite de vitesse et le nombre de rapport.

* **conso_urb**: Consommation urbaine de carburant (en l/100km),

* **conso_exurb**: consommation extra urbaine de carburant (en l/100km),

* **conso_mixte**: Consommation mixte de carburant (en l/100km),

* **co2**: Emission de CO2 (en g/km),

* **masse_ordma_min**: Masse en ordre de marche mini

* **masse_ordma_max**: Masse en ordre de marche max
 
* **Carrosserie**: Carrosserie

* **gamme**: Gamme du véhicule



# Objectif

Notre objectif majeur dans ce notebook 2 est de :

Prédire les emisisions de **CO2** des vehicules en fonction de certaines informations (Variables explicatives)

* En utilisant 4 à 5 modéles différents

* En comparant  les scores
    
* En choissisant le meilleur modèle


# Description des données

Lien vers les données: https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/


### Chargement, lecture, apercu et infos des données

In [142]:
data_model = pd.read_csv("data_model.csv")
data_model['Carrosserie'].to_csv('carrosseries.csv')


201.67065857381698


### Selectionner les feautures les plus importans

In [143]:
New_Data = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]

In [144]:
Y = New_Data['co2']
X = New_Data.drop(['co2'] ,axis =1)

### Traiter la colonne Carrosserie

In [145]:
le = LabelEncoder()
X['Carrosserie']=le.fit_transform(X['Carrosserie'])
dump(le,'Encoder.pkl')

['Encoder.pkl']

### Prediction de CO2
    
Pour chacun de nos modèles:
   * **DummyRegressor**,
   * **LinearRegression**, 
   * **SGDRegressor**, 
   * **RandomForestRegressor**,
   * **GradientBoostingRegressor**,
   * **XGBRegressor**, 
    
Nous allons predire le niveau d'émission de **Co2** puis établir une comparaison entre les performances de chaque modèle.

    Appliquer un GridSearch pour optimiser les hyperparamètres de chaque modèle
    Paramètrer dans GridSearch les scores qui sont adaptés à la régression
    Identifier le meilleur modèle

In [146]:
model = [DummyRegressor(),LinearRegression(),SGDRegressor(),RandomForestRegressor(),GradientBoostingRegressor(), XGBRegressor()]
param_grids = [{'strategy':["mean", "median"]},
               {"fit_intercept":[True, False]},
               {'penalty':['l2','l1','elasticnet',None]},
               {'n_estimators':[25,50,100], 'criterion':['squared_error']},
               {'loss':['squared_error',],'learning_rate':[0.1,1]},
               {}]

results = []

scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

dump(scaler, 'Scaler.pkl')

for i, (model, param_grid) in enumerate(zip(model, param_grids)):
     grid = GridSearchCV(model,param_grid, refit= True, n_jobs=-1)
     grid.fit(X,Y)
     results.append({'model':model, 'best_params':grid.best_params_, 'score':grid.best_score_})
for i in results:
     print(i)
    

{'model': DummyRegressor(), 'best_params': {'strategy': 'median'}, 'score': -0.21342788285930653}
{'model': LinearRegression(), 'best_params': {'fit_intercept': True}, 'score': 0.15675550225254772}
{'model': SGDRegressor(), 'best_params': {'penalty': 'l2'}, 'score': 0.16328320517308184}
{'model': RandomForestRegressor(), 'best_params': {'criterion': 'squared_error', 'n_estimators': 100}, 'score': 0.24728143728038648}
{'model': GradientBoostingRegressor(), 'best_params': {'learning_rate': 0.1, 'loss': 'squared_error'}, 'score': 0.2988379974139376}
{'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_

In [147]:
closest_to_zero_score = float('inf')

best_params = None
best_model_name = None

for result in results:
    score = result['score']
    model = result['model']
    if abs(score) < abs(closest_to_zero_score):
        closest_to_zero_score = score
        best_params = result['best_params']
        best_model = model

In [148]:
model=best_model.set_params(**best_params)
model.fit(X,Y)

dump(model,'Model.pkl')

['Model.pkl']

### Application Web

Développer une application Streamlit avec les options suivantes :

    1- L'utilisateur doit selectionner le type du carrosserie à partire d'un menu déroulant
    2- L'utilisateur doit saisir 'masse_ordma_min' et 'masse_ordma_max' dans deux champs de saisie différent
    3- Programmer un boutton pour lancer la prédiction de CO2